Adopted from https://www.kaggle.com/code/yxyyxy/rsna2024-training-baseline-2nd-stage/edit

# RSNA2024 LSDC Submission Baseline

This notebook will Let the model infer and make a submission.

### My other Notebooks
- [RSNA2024 LSDC Making Dataset](https://www.kaggle.com/code/itsuki9180/rsna2024-lsdc-making-dataset) 
- [RSNA2024 LSDC Training Baseline](https://www.kaggle.com/code/itsuki9180/rsna2024-lsdc-training-baseline) 
- [RSNA2024 LSDC Submission Baseline](https://www.kaggle.com/code/itsuki9180/rsna2024-lsdc-submission-baseline) <- you're reading now

# Import Libralies

In [1]:
!python /kaggle/input/script-deepspine-custom-dataset/main.py \
'/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_series_descriptions.csv' \
'/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv' \
'/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images' \
'/kaggle/input/2d-segmentation-of-sagittal-lumbar-spine-mri/simple_unet.pth'

Using path: /kaggle/input/2d-segmentation-of-sagittal-lumbar-spine-mri/simple_unet.pth
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using path: /kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_series_descriptions.csv
Using base path: /kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images
Processing studies: 100%|█████████████████████████| 1/1 [00:08<00:00,  8.09s/it]
Pipeline completed successfully!
Processing studies: 100%|█████████████████████████| 1/1 [00:00<00:00,  1.10it/s]
Axial dat

In [2]:
import os
import gc
import sys
from PIL import Image
import cv2
import math, random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

from collections import OrderedDict

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset, Sampler
from torch.optim import AdamW

import timm
from transformers import get_cosine_schedule_with_warmup

import albumentations as A


In [3]:
rd = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification'

# Define the directory path
DATA_fromStage1 = '/kaggle/working'


# Config

In [4]:
OUTPUT_DIR = f'/kaggle/input/rsna2024-lsdc-training-baseline/rsna24-results'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
N_WORKERS = os.cpu_count()
USE_AMP = True
SEED = 1

IMG_SIZE = [225, 225]
N_LABELS = 5
N_CLASSES = 3 * N_LABELS

model_name_sag = 'efficientnet_b0'
model_name_axi = 'resnet34'
# resnet34
in_chans_sag = 30
in_chans_axi = 4

N_FOLDS = 5

BATCH_SIZE = 1

In [5]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda', index=0)

In [6]:
CONDITIONS = [
    'spinal_canal_stenosis', 
    'left_neural_foraminal_narrowing', 
    'right_neural_foraminal_narrowing',
    'left_subarticular_stenosis',
    'right_subarticular_stenosis'
]

# Define the mapping for each level
level_mapping = {
    'L1/L2': 'l1_l2',
    'L2/L3': 'l2_l3',
    'L3/L4': 'l3_l4',
    'L4/L5': 'l4_l5',
    'L5/S1': 'l5_s1'
}


In [7]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [8]:
# Define the path to CSV file
csv_file_path = DATA_fromStage1 + '/sagittalT2/dataset_metadata.csv'
# Read the CSV file
dataset_metadata = pd.read_csv(csv_file_path)
dataset_metadata.rename(columns={'Unnamed: 0': 'st_id'}, inplace=True)
dataset_metadata.set_index('st_id', inplace=True)

# Print the number of columns and rows in the DataFrame
num_rows = dataset_metadata.shape[0]
print(f"columns: {dataset_metadata.columns}")
print(f"Number of rows: {num_rows}")

# print(dataset_metadata.head())
# Stack the 2nd to 6th columns into a single column
stacked_df = dataset_metadata.stack(dropna=False).reset_index(level=1)
stacked_df.columns = ['Level', 'fn']
# stacked_df = stacked_df.reset_index(drop=True)
print(stacked_df)

print("----------- test index 0 -------------")
st_id = stacked_df.index[0]
row_idx = stacked_df.iloc[0]
print(st_id)
print(row_idx['Level'])
print(row_idx['fn'])
print("----------- test index 1 -------------")
st_id = stacked_df.index[1]
row_idx = stacked_df.iloc[1]
print(st_id)
print(row_idx['Level'])
print(row_idx['fn'])

columns: Index(['L4/L5', 'L3/L4', 'L2/L3', 'L1/L2', 'L5/S1'], dtype='object')
Number of rows: 1
          Level                                             fn
st_id                                                         
44036939  L4/L5  /kaggle/working/sagittalT2/44036939/L4_L5.npy
44036939  L3/L4  /kaggle/working/sagittalT2/44036939/L3_L4.npy
44036939  L2/L3  /kaggle/working/sagittalT2/44036939/L2_L3.npy
44036939  L1/L2  /kaggle/working/sagittalT2/44036939/L1_L2.npy
44036939  L5/S1  /kaggle/working/sagittalT2/44036939/L5_S1.npy
----------- test index 0 -------------
44036939
L4/L5
/kaggle/working/sagittalT2/44036939/L4_L5.npy
----------- test index 1 -------------
44036939
L3/L4
/kaggle/working/sagittalT2/44036939/L3_L4.npy


/tmp/ipykernel_24/3549742913.py:15: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  stacked_df = dataset_metadata.stack(dropna=False).reset_index(level=1)


In [9]:
'''# one npy file example
print("--------------------- SagittalT2 samples -----------")
example_npy_fn = dataset_metadata['L4/L5'][0]
example_npy_fn = example_npy_fn.split('/')
# Load the .npy file
example_npy = np.load(os.path.join(DATA_fromStage1, *(example_npy_fn[-3:])))
# Print the structure of the data
print(f"Shape of the data: {example_npy.shape}")
print(f"Data type: {example_npy.dtype}")
# Show the 15 slices one by one
for i in range(15):
    plt.imshow(example_npy[i], cmap='gray')
    plt.title(f'Slice {i+1}')
    plt.axis('off')
    plt.show()

print("--------------------- SagittalT1 samples -----------")
example_npy = np.load(os.path.join(DATA_fromStage1, 'sagittalT1', *(example_npy_fn[-2:])))
# Print the structure of the data
print(f"Shape of the data: {example_npy.shape}")
print(f"Data type: {example_npy.dtype}")
# Show the 15 slices one by one
for i in range(15):
    plt.imshow(example_npy[i], cmap='gray')
    plt.title(f'Slice {i+1}')
    plt.axis('off')
    plt.show()

print("--------------------- AxialT2 samples ----------")
# Load the .npy file
example_npy = np.load(os.path.join(DATA_fromStage1, 'axialT2', *(example_npy_fn[-2:])))
print(f"Shape of the data: {example_npy.shape}")
print(f"Data type: {example_npy.dtype}")
for i in range(6):
    img = example_npy[i]
    plt.imshow(img, cmap='gray')
    plt.title(f'Slice {i+1}')
    plt.axis('off')
    plt.show()
'''

'# one npy file example\nprint("--------------------- SagittalT2 samples -----------")\nexample_npy_fn = dataset_metadata[\'L4/L5\'][0]\nexample_npy_fn = example_npy_fn.split(\'/\')\n# Load the .npy file\nexample_npy = np.load(os.path.join(DATA_fromStage1, *(example_npy_fn[-3:])))\n# Print the structure of the data\nprint(f"Shape of the data: {example_npy.shape}")\nprint(f"Data type: {example_npy.dtype}")\n# Show the 15 slices one by one\nfor i in range(15):\n    plt.imshow(example_npy[i], cmap=\'gray\')\n    plt.title(f\'Slice {i+1}\')\n    plt.axis(\'off\')\n    plt.show()\n\nprint("--------------------- SagittalT1 samples -----------")\nexample_npy = np.load(os.path.join(DATA_fromStage1, \'sagittalT1\', *(example_npy_fn[-2:])))\n# Print the structure of the data\nprint(f"Shape of the data: {example_npy.shape}")\nprint(f"Data type: {example_npy.dtype}")\n# Show the 15 slices one by one\nfor i in range(15):\n    plt.imshow(example_npy[i], cmap=\'gray\')\n    plt.title(f\'Slice {i+1}\'

# Define Dataset

In [10]:
class RSNA24TestDataset(Dataset):
    def __init__(self, df_fn, Slice_len_Sag=15, Slice_len_Axi=4, transform=None, trainsform_axis=None):
        self.df_fn = df_fn
        self.transform = transform
        self.trainsform_axis = trainsform_axis
        self.Slice_len_Sag = Slice_len_Sag
        self.Slice_len_Axi = Slice_len_Axi
        # Select all rows where the 'Name' column has the value 'Alice'
    def __len__(self):
        return len(self.df_fn)

    def __getitem__(self, idx):
        st_id = stacked_df.index[idx]
        row_idx = stacked_df.iloc[idx]
        level = (row_idx['Level'])
        filename = row_idx['fn']
        
        path_split = filename.split('/')
        # Saggital T2 ------------  Load the .npy file
        npy_sagT2_path = os.path.join(DATA_fromStage1, path_split[3], path_split[4], path_split[5])
        npy_sagT2 = np.load(npy_sagT2_path).astype(np.float32)
        current_length = npy_sagT2.shape[0]
        if current_length > self.Slice_len_Sag:
            indices = np.linspace(0, current_length - 1, self.Slice_len_Sag, dtype=int)
            npy_sagT2 = npy_sagT2[indices, :, :]
        elif current_length < self.Slice_len_Sag:
            indices = np.linspace(0, current_length - 1, self.Slice_len_Sag, dtype=int)
            npy_sagT2 = npy_sagT2[indices, :, :]

        # Saggital T1 ------------  Load the .npy file
        npy_sagT1_path = os.path.join(DATA_fromStage1, "sagittalT1", path_split[4], path_split[5])
        npy_sagT1 = np.load(npy_sagT1_path).astype(np.float32)
        current_length = npy_sagT1.shape[0]
        if current_length > self.Slice_len_Sag:
            indices = np.linspace(0, current_length - 1, self.Slice_len_Sag, dtype=int)
            npy_sagT1 = npy_sagT1[indices, :, :]
        elif current_length < self.Slice_len_Sag:
            indices = np.linspace(0, current_length - 1, self.Slice_len_Sag, dtype=int)
            npy_sagT1 = npy_sagT1[indices, :, :]

        # Axial T2 ------------ Load the .npy file
        npy_AxiT2_path = os.path.join(DATA_fromStage1 , "axialT2", path_split[4], path_split[5])
        npy_AxiT2 = np.load(npy_AxiT2_path).astype(np.float32)
        current_length = npy_AxiT2.shape[0]
        if current_length > self.Slice_len_Axi:
            indices = np.linspace(0, current_length - 1, self.Slice_len_Axi, dtype=int)
            npy_AxiT2 = npy_AxiT2[indices, :, :]
        elif current_length < self.Slice_len_Axi:
            indices = np.linspace(0, current_length - 1, self.Slice_len_Axi, dtype=int)
            npy_AxiT2 = npy_AxiT2[indices, :, :]

        #  npy_sagT2 and npy_AxiT2 are in the format [channels, height, width]        
        npy_sagT1 = np.transpose(npy_sagT1, (1, 2, 0))  # Transpose to [height, width, channels]
        npy_sagT2 = np.transpose(npy_sagT2, (1, 2, 0))  # Transpose to [height, width, channels]
        npy_AxiT2 = np.transpose(npy_AxiT2, (1, 2, 0))  # Transpose to [height, width, channels]

        if self.transform is not None:
            npy_sagT1 = self.transform(image=npy_sagT1)['image']
            npy_sagT2 = self.transform(image=npy_sagT2)['image']
            npy_AxiT2 = self.trainsform_axis(image=npy_AxiT2)['image']

        # transpose back to the original format
        npy_sagT1 = np.transpose(npy_sagT1, (2, 0, 1))
        npy_sagT2 = np.transpose(npy_sagT2, (2, 0, 1))  # Transpose back to [channels, height, width]
        npy_AxiT2 = np.transpose(npy_AxiT2, (2, 0, 1))  # Transpose back to [channels, height, width]

        return st_id, npy_sagT1, npy_sagT2, npy_AxiT2, level


In [11]:
transforms_test_Sag = A.Compose([
    A.Resize(84, 160),
    A.Normalize(mean=0.5, std=0.5)
])

transforms_test = A.Compose([
    A.Resize(IMG_SIZE[0], IMG_SIZE[1]),
    A.Normalize(mean=0.5, std=0.5)
])


In [12]:
test_ds = RSNA24TestDataset(stacked_df, transform=transforms_test_Sag, trainsform_axis=transforms_test)
test_dl = DataLoader(
    test_ds, 
    batch_size=1, 
    shuffle=False,
    num_workers=N_WORKERS,
    pin_memory=True,
    drop_last=False
)


In [13]:
from collections import Counter

sagT2_slicenum = []
AxiT2_slicenum = []

print(test_dl.__len__())

# Iterate through the data loader and append slice numbers
for idx, (st_id, npy_sagT1, npy_sagT2, npy_AxiT2, level) in enumerate(test_dl):
    sagT2_slicenum.append(npy_sagT2.shape)
    AxiT2_slicenum.append(npy_AxiT2.shape)
    assert not torch.isnan(npy_sagT2).any(), "NaN values found in npy_sagT2"
    assert not torch.isnan(npy_AxiT2).any(), "NaN values found in npy_AxiT2"
    print(level)

# Count the occurrences of each unique value in the lists
sagT2_counts = Counter(sagT2_slicenum)
AxiT2_counts = Counter(AxiT2_slicenum)

print("Occurrences of each unique value in sagT2_slicenum:")
for value, count in sagT2_counts.items():
    print(f"Value: {value}, Count: {count}")

print("Occurrences of each unique value in AxiT2_slicenum:")
for value, count in AxiT2_counts.items():
    print(f"Value: {value}, Count: {count}")


5
['L4/L5']
['L3/L4']
['L2/L3']
['L1/L2']
['L5/S1']
Occurrences of each unique value in sagT2_slicenum:
Value: torch.Size([1, 15, 84, 160]), Count: 5
Occurrences of each unique value in AxiT2_slicenum:
Value: torch.Size([1, 4, 225, 225]), Count: 5


# Define Model

In [14]:
class LevelHead(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(LevelHead, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class RSNA24Model_Hybrid(nn.Module):
    def __init__(self, model_name_sag, model_name_axi, in_chans_sag, in_chans_axi, num_classes, level_names):
        super(RSNA24Model_Hybrid, self).__init__()
        self.model_sag = timm.create_model(model_name_sag, in_chans=in_chans_sag, global_pool='avg'
                                           , pretrained=False, features_only=False)
        self.model_axi = timm.create_model(model_name_axi, in_chans=in_chans_axi, global_pool='avg'
                                           , pretrained=False, features_only=False)
        
        # Replace the last layer with an identity layer
        if hasattr(self.model_sag, 'classifier'):
            self.model_sag.classifier = nn.Identity()
        elif hasattr(self.model_sag, 'fc'):
            self.model_sag.fc = nn.Identity()
        
        if hasattr(self.model_axi, 'classifier'):
            self.model_axi.classifier = nn.Identity()
        elif hasattr(self.model_axi, 'fc'):
            self.model_axi.fc = nn.Identity()
        
        # Get the output feature sizes
        with torch.no_grad():
            sample_input_sag = torch.randn(1, in_chans_sag, 84, 160)
            sample_input_axi = torch.randn(1, in_chans_axi, 224, 224)
            output_sag = self.model_sag(sample_input_sag)
            output_axi = self.model_axi(sample_input_axi)
        
        # Define the final fully connected layers for each task
        self.fc_heads = nn.ModuleDict({
            level: LevelHead(output_sag.shape[1] + output_axi.shape[1], num_classes) for level in level_names
        })
        
    def forward(self, x_sag, x_axi, level):
        x_sag = self.model_sag(x_sag)
        x_axi = self.model_axi(x_axi)
        x = torch.cat((x_sag, x_axi), dim=1)
        x = self.fc_heads[level[0]](x) # the input level is a array associated with batch size
        return x


# Load Models

In [15]:
import glob
CKPT_PATHS = glob.glob('/kaggle/input/tpu-rsna2024-training-baseline-2nd-stage/rsna24-results/best_wll_model_fold-*.pt')
CKPT_PATHS = sorted(CKPT_PATHS)


In [16]:
# Ensure device is set correctly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models = []
for i, cp in enumerate(CKPT_PATHS):
    print(f'loading {cp}...')
    model = RSNA24Model_Hybrid(model_name_sag, model_name_axi,
                               in_chans_sag, in_chans_axi, 
                               num_classes=N_CLASSES, level_names=level_mapping.keys())
    model.load_state_dict(torch.load(cp, map_location=device))
    model.eval()
    model.to(device)
    models.append(model)

print(models)
    

loading /kaggle/input/tpu-rsna2024-training-baseline-2nd-stage/rsna24-results/best_wll_model_fold-0.pt...
loading /kaggle/input/tpu-rsna2024-training-baseline-2nd-stage/rsna24-results/best_wll_model_fold-1.pt...
loading /kaggle/input/tpu-rsna2024-training-baseline-2nd-stage/rsna24-results/best_wll_model_fold-2.pt...
loading /kaggle/input/tpu-rsna2024-training-baseline-2nd-stage/rsna24-results/best_wll_model_fold-3.pt...
loading /kaggle/input/tpu-rsna2024-training-baseline-2nd-stage/rsna24-results/best_wll_model_fold-4.pt...
[RSNA24Model_Hybrid(
  (model_sag): EfficientNet(
    (conv_stem): Conv2d(30, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), paddi

# Inference loop

In [17]:
autocast = torch.cuda.amp.autocast(enabled=USE_AMP, dtype=torch.half)
y_preds = []
row_names = []

with tqdm(test_dl, leave=True) as pbar:
    with torch.no_grad():
        for idx, (st_id, npy_sagT1, npy_sagT2, npy_AxiT2, level) in enumerate(pbar):
            npy_sagT1 = npy_sagT1.to(device)
            npy_sagT2 = npy_sagT2.to(device)
            npy_AxiT2 = npy_AxiT2.to(device)
            pred_per_study = np.zeros((5, 3))  
            with torch.cuda.amp.autocast(): 
                for m in models:    
                    y = m(torch.cat((npy_sagT1, npy_sagT2), axis=1), npy_AxiT2, level)[0]
                    for col in range(N_LABELS):
                        pred = y[col*3:col*3+3]
                        y_pred = pred.float().softmax(0).cpu().numpy()
                        pred_per_study[col] += y_pred / len(models)
                y_preds.append(pred_per_study)

            for cond in CONDITIONS:
                row_name = f"{str(st_id.item())}_{cond}_{level_mapping[''.join(level)]}"
                row_names.append(row_name)

y_preds = np.concatenate(y_preds, axis=0)
                    
print(len(row_names))
print(y_preds)
print(y_preds.shape)

100%|██████████| 5/5 [00:01<00:00,  3.34it/s]

25
[[0.33048242 0.21228901 0.45722856]
 [0.29005859 0.43338696 0.27655445]
 [0.28137716 0.42492159 0.29370125]
 [0.17375995 0.31870165 0.50753841]
 [0.17551581 0.28116985 0.54331434]
 [0.35552517 0.28761901 0.35685583]
 [0.37917938 0.40453798 0.21628261]
 [0.38986821 0.4057321  0.2043997 ]
 [0.25414692 0.35101504 0.39483805]
 [0.2589484  0.31709433 0.42395724]
 [0.57228702 0.23968393 0.18802905]
 [0.62000209 0.29076234 0.08923557]
 [0.6217688  0.28954596 0.08868524]
 [0.44202141 0.3119198  0.24605875]
 [0.46662565 0.32474605 0.2086283 ]
 [0.49058041 0.25051424 0.25890533]
 [0.54034166 0.27185891 0.18779943]
 [0.49853682 0.27392853 0.22753465]
 [0.43879917 0.3011656  0.26003521]
 [0.4195095  0.33840711 0.24208338]
 [0.6841981  0.16860797 0.14719393]
 [0.29502338 0.36020254 0.34477406]
 [0.32609934 0.3385221  0.33537853]
 [0.37124861 0.32402177 0.30472961]
 [0.35404973 0.30537765 0.34057258]]
(25, 3)


In [18]:
sample_sub = pd.read_csv(f'{rd}/sample_submission.csv')
print(sample_sub)

LABELS = list(sample_sub.columns[1:])
LABELS

                                             row_id  normal_mild  moderate  \
0    44036939_left_neural_foraminal_narrowing_l1_l2     0.333333  0.333333   
1    44036939_left_neural_foraminal_narrowing_l2_l3     0.333333  0.333333   
2    44036939_left_neural_foraminal_narrowing_l3_l4     0.333333  0.333333   
3    44036939_left_neural_foraminal_narrowing_l4_l5     0.333333  0.333333   
4    44036939_left_neural_foraminal_narrowing_l5_s1     0.333333  0.333333   
5         44036939_left_subarticular_stenosis_l1_l2     0.333333  0.333333   
6         44036939_left_subarticular_stenosis_l2_l3     0.333333  0.333333   
7         44036939_left_subarticular_stenosis_l3_l4     0.333333  0.333333   
8         44036939_left_subarticular_stenosis_l4_l5     0.333333  0.333333   
9         44036939_left_subarticular_stenosis_l5_s1     0.333333  0.333333   
10  44036939_right_neural_foraminal_narrowing_l1_l2     0.333333  0.333333   
11  44036939_right_neural_foraminal_narrowing_l2_l3     0.333333

['normal_mild', 'moderate', 'severe']

# Make Submission

In [19]:
sub = pd.DataFrame()
sub['row_id'] = row_names
sub[LABELS] = y_preds
sub.head(25)

,row_id,normal_mild,moderate,severe
0,44036939_spinal_canal_stenosis_l4_l5,0.330482,0.212289,0.457229
1,44036939_left_neural_foraminal_narrowing_l4_l5,0.290059,0.433387,0.276554
2,44036939_right_neural_foraminal_narrowing_l4_l5,0.281377,0.424922,0.293701
3,44036939_left_subarticular_stenosis_l4_l5,0.173760,0.318702,0.507538
4,44036939_right_subarticular_stenosis_l4_l5,0.175516,0.281170,0.543314
5,44036939_spinal_canal_stenosis_l3_l4,0.355525,0.287619,0.356856
6,44036939_left_neural_foraminal_narrowing_l3_l4,0.379179,0.404538,0.216283
7,44036939_right_neural_foraminal_narrowing_l3_l4,0.389868,0.405732,0.204400
8,44036939_left_subarticular_stenosis_l3_l4,0.254147,0.351015,0.394838
9,44036939_right_subarticular_stenosis_l3_l4,0.258948,0.317094,0.423957


In [20]:
sub.to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head()

,row_id,normal_mild,moderate,severe
0,44036939_spinal_canal_stenosis_l4_l5,0.330482,0.212289,0.457229
1,44036939_left_neural_foraminal_narrowing_l4_l5,0.290059,0.433387,0.276554
2,44036939_right_neural_foraminal_narrowing_l4_l5,0.281377,0.424922,0.293701
3,44036939_left_subarticular_stenosis_l4_l5,0.173760,0.318702,0.507538
4,44036939_right_subarticular_stenosis_l4_l5,0.175516,0.281170,0.543314


# Conclusion
We created the dataset, performed training, and inference in this notebook. 

This competition is a bit complicated to handle the dataset, so there may be a better way.

I think there are many other areas to improve in my notebook. I hope you can learn from my notebook and get a better score.